#### Week 3 Web Scraping Assignment

#### Andrew D. 7/29/20


In [65]:
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from pandas import DataFrame

#### Store the wikipedia URL in the object URL
#### read in the pandas dataframe using pd.read_html and set the axis = 0

In [69]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### We see there are several rows which contain "Not Assigned" which provides no value, so we will remove them from the dataframe

In [70]:
df = df.set_index("Borough")
df = df.drop(["Not assigned"], axis = 0)

df

,Postal Code,Neighbourhood
Borough,,
North York,M3A,Parkwoods
North York,M4A,Victoria Village
Downtown Toronto,M5A,"Regent Park, Harbourfront"
North York,M6A,"Lawrence Manor, Lawrence Heights"
Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government"
...,...,...
Etobicoke,M8X,"The Kingsway, Montgomery Road, Old Mill North"
Downtown Toronto,M4Y,Church and Wellesley
East Toronto,M7Y,"Business reply mail Processing Centre, South C..."


In [71]:
df.shape

(103, 2)

#### Next I will quickly check there are no 'NAN' values in the dataframe 

In [72]:
df.isnull().values.any()

False

#### Import the the coordinates of the locations in the dataframe above 
#### for reproducability the dataset can be found at https://cocl.us/Geospatial_data

In [83]:
df2 = pd.read_csv("C:/Users/thech/Desktop/Geospatial_Coordinates.csv")
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


#### Join the two dataframe on their postal code 

#### Join the two dataframes on their key columns by setting the postal code as the key

In [99]:
df.set_index('Postal Code').join(df2.set_index('Postal Code'))
df = df.join(df2.set_index('Postal Code'), on = 'Postal Code')


In [122]:
df

,Postal Code,Neighbourhood,Latitude,Longitude
Borough,,,,
North York,M3A,Parkwoods,43.753259,-79.329656
North York,M4A,Victoria Village,43.725882,-79.315572
Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
North York,M6A,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
Etobicoke,M8X,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
Downtown Toronto,M4Y,Church and Wellesley,43.665860,-79.383160
East Toronto,M7Y,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


#### Import the necessary packages to carry out cluster analysis on GIS data

In [90]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium as fl

#### Generate a map of the area using the coordinates from the dataset above

In [135]:
Latitude = 43.6532 
Longitude = -79.3832 

maps = folium.Map(location = [Latitude, Longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng]    ,
        radius = 4    ,
        popup = label ,
        color = 'navy',
        fill  = True  ,
        fill_color = 'mediumblue',
        fill_opacity = .7,
        parse_html = False).add_to(maps)  
    
maps